# LLM API Testing

## Validate game moves
### Two checks to validate the move:
### 1. Check that the square is valid to move (it's not surrounded by other squares)
### 2. Check to make sure the change results in a contiguous shape

In [10]:
# Define a function to check that a square is valid to move.
# The function will take in the 10x10 grid and the coordinates of the square to be moved.
def is_valid_square(grid, coordinates):
    adjacent_squares = [
        coordinates[0] - 1, coordinates[1],
        coordinates[0] + 1, coordinates[1],
        coordinates[0], coordinates[1] - 1,
        coordinates[0], coordinates[1] + 1
    ]

    if grid[coordinates[0]][coordinates[1]] == 1: # First check to make sure the coordinates correspond to a 1
        for pos in adjacent_squares: # If so, then check if any of the adjacent squares are 0
            if grid[adjacent_squares[0]][adjacent_squares[1]] == 0:
                return True
    return False

# Example usage
grid = [
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
]
coordinates = (1,5)
print(is_valid_square(grid, coordinates)) 

True


In [17]:

    
# Define a function for check 2 using DFS.
# It takes in the 10x10 grid and the coordinates of a starting point on the shape.
# If the count = 10, the function will return True, indicating that the move is valid.
def is_contiguous(grid, start):
    rows = len(grid)
    cols = len(grid[0])
    visited = set()
    
    def dfs(r, c):
        if (r, c) in visited or not (0 <= r < rows and 0 <= c < cols) or grid[r][c] == 0:
            return 0
        visited.add((r, c))
        count = 1 
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]: # Check all 4 directions
            count += dfs(r + dr, c + dc)
        return count
    
    return dfs(start[0], start[1]) == 10

# Example usage:
grid = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
]
start = (1, 3)
print(is_contiguous(grid, start))  
    



True


In [18]:
# Define a function to check that there is exactly one change between the previous shape and the new shape.

def is_one_change(grid1, grid2):
    count = 0
    for i in range(len(grid1)):
        for j in range(len(grid1[0])):
            if grid1[i][j] != grid2[i][j]:
                count += 1
    return count == 1

# Example usage:
grid1 = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
]
grid2 = [     
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
]
print(is_one_change(grid1, grid2)) 

False


# Get response from LLM

In [3]:
import json
import datetime



# Set your OpenAI API key
# import openai
# openai.api_key = 'XXXXXX'

# Define a function to get a response from the OpenAI API
def get_openai_response_fake(user_prompt, prompt):
    # use user_prompt as file_name to get the full game instructions
    try:
        prompt = {k:v for k,v in prompt.items() if v}  # Remove empty values from prompt (e.g., initial prompt)
        input = {
            'user_prompt': user_prompt,    # Full game instructions
            'prompt': json.dumps(prompt),  # Current move instructions (dict dumped to string)
        }

        # Todo: actually call API
        response = "2 1022"  # Hardcoded fake response
        output = input.copy()
        output['current_shape'] = prompt.get('current_shape')
        output['last_shape'] = prompt.get('last_shape')
        output['response'] = response
        output['timestamp'] = datetime.datetime.now().isoformat()
        output['valid_move'] = True # TODO Hardcoded, but need to write a function for checking
        return output
    except Exception as e:
        return f"An error occurred: {e}"
    
# Valid moves example
# 1023 -> 2 1022 -> 4 1020 512

# Single example
user_prompt = "instructions/version1.txt"
prompt={
    "current_shape": "1023", 
    "last_shape": ""
}
get_openai_response_fake(user_prompt, prompt)

{'user_prompt': 'instructions/version1.txt',
 'prompt': '{"current_shape": "1023"}',
 'current_shape': '1023',
 'last_shape': None,
 'response': '2 1022',
 'timestamp': '2025-01-30T14:03:08.350019',
 'valid_move': True}

In [4]:
# Interaction loop for game

THRESHOLD_TOTAL_MOVES = 10
THRESHOLD_TOTAL_RETRIES = 10

user_prompt = "instructions/version1.txt"
prompt={
    "current_shape": "1023", 
    "last_shape": ""
}

move_count = 0
retry_count = 0

while move_count < THRESHOLD_TOTAL_MOVES and retry_count < THRESHOLD_TOTAL_RETRIES:
    output = get_openai_response_fake(user_prompt, prompt)
    print(output)
    if output['valid_move']:
        move_count += 1
        retry_count = 0
        prompt['current_shape'] = output['response']
    else:
        retry_count += 1
        print(f"Invalid move, retrying... {retry_count} / {THRESHOLD_TOTAL_RETRIES}")
    
    # TODO: save response dictionary to file (append as you go)


{'user_prompt': 'instructions/version1.txt', 'prompt': '{"current_shape": "1023"}', 'current_shape': '1023', 'last_shape': None, 'response': '2 1022', 'timestamp': '2025-01-30T14:03:08.384481', 'valid_move': True}
{'user_prompt': 'instructions/version1.txt', 'prompt': '{"current_shape": "2 1022"}', 'current_shape': '2 1022', 'last_shape': None, 'response': '2 1022', 'timestamp': '2025-01-30T14:03:08.384932', 'valid_move': True}
{'user_prompt': 'instructions/version1.txt', 'prompt': '{"current_shape": "2 1022"}', 'current_shape': '2 1022', 'last_shape': None, 'response': '2 1022', 'timestamp': '2025-01-30T14:03:08.385012', 'valid_move': True}
{'user_prompt': 'instructions/version1.txt', 'prompt': '{"current_shape": "2 1022"}', 'current_shape': '2 1022', 'last_shape': None, 'response': '2 1022', 'timestamp': '2025-01-30T14:03:08.385114', 'valid_move': True}
{'user_prompt': 'instructions/version1.txt', 'prompt': '{"current_shape": "2 1022"}', 'current_shape': '2 1022', 'last_shape': None,